<img src="https://valueguard.se/static/media/valueguardlogo_black.f3a4c174.png" alt="logo" width="400"/>

# Residential Valuations

* Based on official data from the Swedish Land Registry (_Lantmäteriet_), but managed and improved by Valueguard and available through our easy to use API.
* The residential registry contains more than 5.6 million homes in Sweden, of which **2.1 million** are houses and **2.7 million** are apartments. 
* We have correctly coded **85** percent of all tenancies and **99** percent of all cooperative housing.
* The variable "construction_year" is missing on **9** percent of houses (2-4 percent missing for houses built 1980 and earlier) and **30** percent of apartments, a total of **22** percent of all homes
* We update the registry every quarter
* The registry is complete with number of rooms and kitchen type for apartments
* The matching percentage of sold homes to the registry is **94.3** percent for sales within the last 12 months and **97.5** percent for all older sales

# Install dependencies

In [83]:
!pip3 install git+https://github.com/Valueguard-Index-Sweden/valueguard-python-client#egg=valueguard
!pip3 install geopandas
!pip3 install folium

  Cloning https://github.com/Valueguard-Index-Sweden/valueguard-python-client to /tmp/pip-install-3vi5kdyt/valueguard_c4ec9f4d751e4340826b5a721e6f3dc4
  Running command git clone -q https://github.com/Valueguard-Index-Sweden/valueguard-python-client /tmp/pip-install-3vi5kdyt/valueguard_c4ec9f4d751e4340826b5a721e6f3dc4


## Imports

In [84]:
import os
import valueguard
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

## Settings

In [85]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Authenticate 

In [86]:
# Create object
vgClient = valueguard.Client()

valueguard_api_username = os. environ['VALUEGUARD_API_USERNAME']
valueguard_api_password = os. environ['VALUEGUARD_API_PASSWORD']

# Generate a access token
vgClient.authenticate(valueguard_api_username,valueguard_api_password)

## Parameters

In [87]:
residential_registry_parameters = {
    "id":"",
    "address":"vangsbyvägen",
    "apartment_area_max":"",
    "apartment_area_min":"",
    "apartment_category":"",
    "apartment_nr":"",
    "area_id":"",
    "building_purpose":"",
    "construction_year_max":"",
    "construction_year_min":"",
    "floor_max":"",
    "floor_min":"",
    "floors_max":"",
    "floors_min":"",
    "kitchen_type":"",
    "living_space_max":"",
    "living_space_min":"",
    "owner_id":"",
    "postal_area":"",
    "postal_code":"",
    "rooms_max":"",
    "rooms_min":"",
    "updated_at_min":"",
    "street":"",
    "letter":"",
    "number":"",
    "property_key":"",
    "ownership_type":"",
    "building_type":"",
    "county_code":"",
    "municipality_code":"",
    "parish_code":""
}

limit = 10000
i = 0

# Get data

In [88]:
search_criteria={}

for parameter_name in residential_registry_parameters:
    if residential_registry_parameters[parameter_name] != "":
        search_criteria[parameter_name] = residential_registry_parameters[parameter_name]

residential_registry_df = pd.DataFrame()

while True:
    data = vgClient.residential_registry(offset=i, limit=limit,search_criteria=search_criteria)
    residential_registry_df = residential_registry_df.append(pd.DataFrame(data['residences']))
    fetched_data = data['meta_data']['offset'] + data['meta_data']['response_rows']
    if data['meta_data']['total_nr_records'] <= data['meta_data']['offset'] + data['meta_data']['response_rows']:
        break
    i += data['meta_data']['response_rows']

In [89]:
residential_registry_df[:5]

,area_name,building_purpose,rooms,rt90x,county_code,ownership_type,owner_id,municipality_code,property_key,area_id,vacation_like_home,apartment_category,number,floors,updated_at,street,wgs84lat,home_type,id,floor,postal_area,sweref99x,sweref99y,address,updated_from,apartment_nr,parish_code,person_registered,probability_deleted,construction_year,deleted,kitchen_type,wgs84lon,apartment_area,letter,error_code,house_nr,rt90y,postal_code,type_code,building_type
0,Uppsala__Vänge/Börje/Uppsala närområde västra,1,NaN,6639042.0,03,äganderätt,,80,030074683,6616,0,NaN,17,NaN,2021-02-22,Vangsbyvägen,59.860649,villa,2308840,NaN,Vänge,6638403,636486,"Vangsbyvägen 17, 75576",public_data,NaN,14,1,0.0,1953,0,NaN,17.437088,178,,0,1,1591440.0,75576,220.0,30
1,Uppsala__Vänge/Börje/Uppsala närområde västra,1,NaN,6639128.0,03,äganderätt,,80,030074684,6616,0,NaN,150,NaN,2021-02-22,Vangsbyvägen,59.861411,lantbruk,2308842,NaN,Vänge,6638490,636540,"Vangsbyvägen 150, 75576",public_data,NaN,14,1,0.0,1900,0,NaN,17.438109,166,,0,1,1591495.0,75576,120.0,30
2,Uppsala__Vänge/Börje/Uppsala närområde västra,1,NaN,6638768.0,03,äganderätt,,80,030074691,6616,0,NaN,2,NaN,2021-02-22,Vangsbyvägen,59.858261,villa,2308831,NaN,Vänge,6638125,636165,"Vangsbyvägen 2, 75576",public_data,NaN,14,1,0.0,1956,0,NaN,17.431180,141,,0,1,1591116.0,75576,220.0,30
3,Uppsala__Vänge/Börje/Uppsala närområde västra,1,NaN,6638811.0,03,äganderätt,,80,030074693,6616,0,NaN,8,NaN,2021-02-22,Vangsbyvägen,59.858641,villa,2308836,NaN,Vänge,6638169,636210,"Vangsbyvägen 8A, 75576",public_data,NaN,14,1,0.0,2003,0,NaN,17.432012,94,A,0,1,1591161.0,75576,220.0,30
4,Uppsala__Vänge/Börje/Uppsala närområde västra,1,NaN,6638948.0,03,äganderätt,,80,030076000,6616,0,NaN,15,NaN,2021-02-22,Vangsbyvägen,59.859816,villa,2308839,NaN,Vänge,6638309,636453,"Vangsbyvägen 15, 75576",public_data,NaN,14,1,0.0,1954,0,NaN,17.436438,162,,0,1,1591406.0,75576,220.0,30


In [90]:
def set_valuation(row):
    valuation_value = None
    valuation = vgClient.residential_registry_valuations(offset=0, limit=1, search_criteria={"id":row['id']})['residences']
    if len(valuation) > 0:
        valuation_values = valuation[0]['valuations']
        if len(valuation_values) > 0:
            valuation_value = valuation_values[0]['value']
        
    return valuation_value 

residential_registry_df['valuation_value'] = residential_registry_df.apply(lambda row : set_valuation(row), axis = 1)

## Remove homes without valuation

In [91]:
residential_registry_heat_map_df = residential_registry_df[residential_registry_df['valuation_value'].notnull()]

## Value plot

In [98]:
m_1 = folium.Map(location=[residential_registry_heat_map_df.iloc[0]['wgs84lat'],residential_registry_df.iloc[0]['wgs84lon']], tiles='openstreetmap', zoom_start=16)

valuation_value_mean = residential_registry_heat_map_df["valuation_value"].mean() 
valuation_value_min = residential_registry_heat_map_df["valuation_value"].min()
valuation_value_max = residential_registry_heat_map_df["valuation_value"].max()

def color_producer(val):
    if val <= valuation_value_mean - (valuation_value_mean - valuation_value_min)/2:
        return 'forestgreen'
    elif val >=  valuation_value_mean + (valuation_value_max - valuation_value_mean)/2:
        return 'darkred'
    else:
        return 'darkorange'

# Add a bubble map to the base map
for i in range(0,len(residential_registry_heat_map_df)):
    Circle(
        location=[residential_registry_heat_map_df.iloc[i]['wgs84lat'], residential_registry_heat_map_df.iloc[i]['wgs84lon']],
        radius=5,
        color=color_producer(residential_registry_heat_map_df.iloc[i]['valuation_value'])).add_to(m_1)

# Display the map
m_1